In [1]:
# installing needed packages
#!pip install tensorflow_hub
#!pip install "tf-models-official==2.15.*"
#!pip install "tensorflow-text==2.15.*"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.7 MB/s eta 0:00:00:00:01
  Attempting uninstall: keras
    Found existing installation: keras 3.0.5
    Uninstalling keras-3.0.5:
      Successfully uninstalled keras-3.0.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 63.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 4.3 MB/s eta 

In [2]:
# imports
import tensorflow as tf
import tensorflow_text as ttext
import tensorflow_hub as thub
import pandas as pd
import numpy as np

2024-03-22 15:08:18.628067: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-22 15:08:18.628168: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-22 15:08:18.749928: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# confirm to see if notebook is using/has GPU
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
# load dataset
trainData = pd.read_csv('/kaggle/input/entity-text-dataset/twitter_training.csv')
testData = pd.read_csv('/kaggle/input/entity-text-dataset/twitter_validation.csv')

In [5]:
# check to see Dataframe
trainData.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [6]:
# check to see Dataframe
testData.head()

,3364,Facebook,Irrelevant,"I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣"
0,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
1,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
2,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
3,4433,Google,Neutral,Now the President is slapping Americans in the...
4,6273,FIFA,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...


In [7]:
# Dataset size
print(trainData.shape)
print(testData.shape)

(74681, 4)
(999, 4)


In [8]:
# Rename Dataset columns
trainData.rename(columns={"Borderlands": "Entity", "Positive": "Sentiment",
                     "im getting on borderlands and i will murder you all ,": "Text"},
                 inplace=True)

testData.rename(columns={"Facebook": "Entity", "Irrelevant": "Sentiment",
                        "I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣": "Text"},
               inplace=True)

In [9]:
# see changes on trainData
trainData.head()

,2401,Entity,Sentiment,Text
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [10]:
# train data on valData
testData.head()

,3364,Entity,Sentiment,Text
0,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
1,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
2,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
3,4433,Google,Neutral,Now the President is slapping Americans in the...
4,6273,FIFA,Negative,Hi @EAHelp I’ve had Madeleine McCann in my cel...


In [11]:
# find any duplicates in datasets
print(trainData.duplicated().sum())
print(testData.duplicated().sum())

2700
0


In [12]:
# get rid of duplicates in trainData
trainData.drop_duplicates(inplace=True)

In [13]:
# see if it worked
trainData.duplicated().sum()

0

In [14]:
# see unique values in trainData and valData
print(trainData.nunique())
print("******************")
print(testData.nunique())

2401         12447
Entity          32
Sentiment        4
Text         69490
dtype: int64
******************
3364         999
Entity        32
Sentiment      4
Text         998
dtype: int64


In [15]:
# concat TEXT and ENTITY values to form one value under TEXT
trainData['Text'] = trainData['Entity'].str.cat(trainData['Text'], sep=" ")
testData['Text'] = testData['Entity'].str.cat(testData['Text'], sep=" ")

In [16]:
# see the changes
trainData.head()

,2401,Entity,Sentiment,Text
0,2401,Borderlands,Positive,Borderlands I am coming to the borders and I w...
1,2401,Borderlands,Positive,Borderlands im getting on borderlands and i wi...
2,2401,Borderlands,Positive,Borderlands im coming on borderlands and i wil...
3,2401,Borderlands,Positive,Borderlands im getting on borderlands 2 and i ...
4,2401,Borderlands,Positive,Borderlands im getting into borderlands and i ...


In [17]:
testData.head()

,3364,Entity,Sentiment,Text
0,352,Amazon,Neutral,Amazon BBC News - Amazon boss Jeff Bezos rejec...
1,8312,Microsoft,Negative,Microsoft @Microsoft Why do I pay for WORD whe...
2,4371,CS-GO,Negative,CS-GO CSGO matchmaking is so full of closet ha...
3,4433,Google,Neutral,Google Now the President is slapping Americans...
4,6273,FIFA,Negative,FIFA Hi @EAHelp I’ve had Madeleine McCann in m...


In [18]:
# drop irrelevant columns
trainData.drop(columns=['2401', 'Entity'], inplace=True)
testData.drop(columns=['3364', 'Entity'], inplace=True)

In [19]:
# see changes
trainData.head()

,Sentiment,Text
0,Positive,Borderlands I am coming to the borders and I w...
1,Positive,Borderlands im getting on borderlands and i wi...
2,Positive,Borderlands im coming on borderlands and i wil...
3,Positive,Borderlands im getting on borderlands 2 and i ...
4,Positive,Borderlands im getting into borderlands and i ...


In [20]:
testData.head()

,Sentiment,Text
0,Neutral,Amazon BBC News - Amazon boss Jeff Bezos rejec...
1,Negative,Microsoft @Microsoft Why do I pay for WORD whe...
2,Negative,CS-GO CSGO matchmaking is so full of closet ha...
3,Neutral,Google Now the President is slapping Americans...
4,Negative,FIFA Hi @EAHelp I’ve had Madeleine McCann in m...


In [21]:
# encode our sentiments
trainData['Sentiment'].replace({'Positive': 1, 'Negative': 0, 'Neutral': 2, 'Irrelevant': 3}, inplace=True)
testData['Sentiment'].replace({'Positive': 1, 'Negative': 0, 'Neutral': 2, 'Irrelevant': 3}, inplace=True)

/tmp/ipykernel_34/1320662073.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  trainData['Sentiment'].replace({'Positive': 1, 'Negative': 0, 'Neutral': 2, 'Irrelevant': 3}, inplace=True)
/tmp/ipykernel_34/1320662073.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  trainData['Sentiment'].replace

In [22]:
# see changes
trainData.head()

,Sentiment,Text
0,1,Borderlands I am coming to the borders and I w...
1,1,Borderlands im getting on borderlands and i wi...
2,1,Borderlands im coming on borderlands and i wil...
3,1,Borderlands im getting on borderlands 2 and i ...
4,1,Borderlands im getting into borderlands and i ...


In [23]:
testData.head()

,Sentiment,Text
0,2,Amazon BBC News - Amazon boss Jeff Bezos rejec...
1,0,Microsoft @Microsoft Why do I pay for WORD whe...
2,0,CS-GO CSGO matchmaking is so full of closet ha...
3,2,Google Now the President is slapping Americans...
4,0,FIFA Hi @EAHelp I’ve had Madeleine McCann in m...


In [24]:
# convert our data to numpy arrays
xData = trainData['Text'].to_numpy()
yData = trainData['Sentiment'].to_numpy()
xTest = testData['Text'].to_numpy()
yTest = testData['Sentiment'].to_numpy()

In [25]:
# imports
import math
num = math.ceil(0.75*71981)

# segment trainData into training and testing dataset
xTrain = xData[:num]
xVal = xData[num:]

yTrain = yData[:num]
yVal = yData[num:]

In [26]:
# perform one-hot encoding for our y values
from sklearn.preprocessing import LabelEncoder

lencoder = LabelEncoder()

# one-hot encoding
yTrain = lencoder.fit_transform(yTrain)
yTest = lencoder.fit_transform(yTest)
yVal = lencoder.fit_transform(yVal)

yTrain = tf.one_hot(yTrain, depth=len(lencoder.classes_))
yTest = tf.one_hot(yTest, depth=len(lencoder.classes_))
yVal = tf.one_hot(yVal, depth=len(lencoder.classes_))

In [27]:
# ensure every value is of type string
xTrain = xTrain.astype(str)
xTest = xTest.astype(str)
xVal = xVal.astype(str)

In [28]:
# links to our pretrained models
bert_preprocessor_link = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
bert_link = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'

In [29]:
# teext sentiment model
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = thub.KerasLayer(bert_preprocessor_link, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = thub.KerasLayer(bert_link, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.2)(net)
  net = tf.keras.layers.Dense(4, activation='softmax', name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [30]:
# just to see if GPU is available for use
if tf.config.list_physical_devices('GPU'):
  print("GPU Available!")
else:
  print("GPU Not Available!")

GPU Available!


In [31]:
# create the model
model = build_classifier_model()

In [32]:
from official.nlp import optimization

epochs = 8
steps_per_epoch = len(xTrain)
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
BERT_optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [33]:
# compile model
model.compile(optimizer=BERT_optimizer, loss=tf.keras.losses.categorical_crossentropy,metrics=['accuracy'])

In [34]:
# set training call-backs
termination_protocol = tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    patience=5,
    restore_best_weights=True
)

In [35]:
# train model
model.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=8, batch_size=30 ,shuffle=True,callbacks=[termination_protocol])

Epoch 1/8
1800/1800 [==============================] - 255s 137ms/step - loss: 1.4372 - accuracy: 0.3442 - val_loss: 1.1888 - val_accuracy: 0.5075
Epoch 2/8
1800/1800 [==============================] - 245s 136ms/step - loss: 1.2058 - accuracy: 0.4794 - val_loss: 1.0311 - val_accuracy: 0.5696
Epoch 3/8
1800/1800 [==============================] - 245s 136ms/step - loss: 1.0578 - accuracy: 0.5610 - val_loss: 0.9177 - val_accuracy: 0.6266
Epoch 4/8
1800/1800 [==============================] - 245s 136ms/step - loss: 0.9547 - accuracy: 0.6113 - val_loss: 0.8306 - val_accuracy: 0.6587
Epoch 5/8
1800/1800 [==============================] - 244s 136ms/step - loss: 0.8496 - accuracy: 0.6587 - val_loss: 0.7399 - val_accuracy: 0.7187
Epoch 6/8
1800/1800 [==============================] - 245s 136ms/step - loss: 0.7320 - accuracy: 0.7114 - val_loss: 0.6387 - val_accuracy: 0.7748
Epoch 7/8
1800/1800 [==============================] - 244s 136ms/step - loss: 0.5999 - accuracy: 0.7666 - val_loss: 0

In [36]:
# evaluate model
model.evaluate(xTest, yTest)

32/32 [==============================] - 2s 73ms/step - loss: 0.5493 - accuracy: 0.8368


[0.549310028553009, 0.8368368148803711]

In [40]:
model.save("sentiment-model.h5")

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [84]:
# test our model on real-life scenarios
# also how the model's output would have 
# been configured if used in production

text = ["COD Good kill.", "FIFA I scored a woldie!", "Amazon Your customer service is appalling!",
       "Facebook You are a sick bastard","Nvidia These are some cool GPUs!",
        "BBC Amad Diallo Scores the winning goal.",
       "Borderlands You can steal and kill this player", "BBC Black people don't deserve anything",
       "NBA2K I made a 3-pointer shot!", "Verizon Your customer service is bad."]

sentiment = model.predict([text])

1/1 [==============================] - 0s 44ms/step


In [85]:
# Results in layman terms
def classify_sentiment(array):
    res = np.argmax(array)
    
    if res == 0:
        return f"Negative with a confedience score = {str(array[res]*100)[:4]}%"
    elif res == 1:
        return f"Positive with a confedience score = {str(array[res]*100)[:4]}%"
    elif res == 2:
        return f"Neutral with a confedience score = {str(array[res]*100)[:4]}%"
    else:
        return f"Irrelevant with a confedience score = {str(array[res]*100)[:4]}%"

# Classify sentiment for each text
sentiment_classes = [classify_sentiment(sentiment[s]) for s in range(len(sentiment))]

for t, s in zip(text, sentiment_classes):
  print(f"| {t} -> {s} |")

| COD Good kill. -> Positive with a confedience score = 92.4% |
| FIFA I scored a woldie! -> Positive with a confedience score = 59.7% |
| Amazon Your customer service is appalling! -> Negative with a confedience score = 89.1% |
| Facebook You are a sick bastard -> Negative with a confedience score = 96.0% |
| Nvidia These are some cool GPUs! -> Positive with a confedience score = 99.8% |
| BBC Amad Diallo Scores the winning goal. -> Irrelevant with a confedience score = 88.2% |
| Borderlands You can steal and kill this player -> Irrelevant with a confedience score = 68.3% |
| BBC Black people don't deserve anything -> Irrelevant with a confedience score = 92.1% |
| NBA2K I made a 3-pointer shot! -> Neutral with a confedience score = 59.6% |
| Verizon Your customer service is bad. -> Negative with a confedience score = 99.2% |
